Задача: вытащить из соответствующего файла базы ФИАС:

1) улицы с нужным названием, добавить к ним регион, м. о., город или населенный пункт;

2) города и населенные пункты

3) муниципальные образования с нужной семантикой и регионом

Регион добавляем из отдельного файла, м.о. и город -- по расщепленному коду ФИАС (plaincode).

На выходе получается 6 датафреймов: 3 фильтрованных и 3 полных

Это отладочный код, на примере Московской области, одной из самых полноводных.

Автоматический код в -- 02_FIAS_street_auto

In [1]:
import pandas as pd
import numpy as np

In [2]:
from dbfread import DBF

### Федеральная информационная адресная система (ФИАС) ###

Единый российский государственный адресный реестр

Источник: ФНС России

Последняя версия: **https://fias.nalog.ru/Updates**

### Описание данных: ###

**ACTSTATUS** - Статус последней исторической записи в жизненном цикле адресного объекта:
- 0 – Не последняя;
- 1 - Последняя

**AOGUID** - глобальный уникальный идентификатор адресного объекта

**AOID** - уникальный идентификатор записи. Ключевое поле.

**AOLEVEL** - уровень адресного объекта
- 1 – регион
- 3 - муниципальный район
- 4 - город
- 6 – населенный пункт
- 7 – улица
- 65 – планировочная структура
- 90 – дополнительные территории (устаревшее)

**AREACODE** - Код района

**AUTOCODE** - Код автономии

**CENTSTATUS** - Статус центра

**CITYCODE** - Код города

**CODE** - Код адресного элемента одной строкой с признаком актуальности из классификационного кода

**CURRSTATUS** - Статус актуальности КЛАДР 4 (последние две цифры в коде)

**ENDDATE** - Окончание действия записи

**FORMALNAME** - Формализованное наименование

**IFNSFL** - Код ИФНС ФЛ

**IFNSUL** - Код ИФНС ЮЛ

**NEXTID** - Идентификатор записи  связывания с последующей исторической записью

**OFFNAME** - Официальное наименование

**OKATO** - ОКАТО

**OKTMO** - ОКТМО

**OPERSTATUS** - Статус действия над записью – причина появления записи (см. OperationStatuses )

**PARENTGUID** - Идентификатор объекта родительского объекта

**PLACECODE** - Код населенного пункта

**PLAINCODE** - Код адресного элемента одной строкой без признака актуальности (последних двух цифр)

**POSTALCODE** - Почтовый индекс

**PREVID** - Идентификатор записи связывания с предыдущей исторической записью

**REGIONCODE** - Код региона

**SHORTNAME** - Краткое наименование типа объекта

**STARTDATE** - Начало действия записи

**STREETCODE** - Код улицы

**TERRIFNSFL** - Код территориального участка ИФНС ФЛ

**TERRIFNSUL** - Код территориального участка ИФНС ЮЛ

**UPDATEDATE** - Дата  внесения (обновления) записи

**CTARCODE** - Код внутригородского района

**EXTRCODE** - Код дополнительного адресообразующего элемента

**SEXTCODE** - Код подчиненного дополнительного адресообразующего элемента

**LIVESTATUS** - Статус актуальности адресного объекта ФИАС на текущую дату:
- 0 – Не актуальный
- 1 - Актуальный
 
**NORMDOC** - Внешний ключ на нормативный документ

**PLANCODE** - Код элемента планировочной структуры

**CADNUM** - Кадастровый номер

**DIVTYPE** - Тип деления:
- 0 – не определено
- 1 – муниципальное
- 2 – административное

### Кодировка региона ###

Код адреса включает:

СС+РРР+ГГГ+ППП+СССС+УУУУ+ДДДД (или ЗЗЗЗ)+ОООО, где:

- **СС** – код субъекта Российской Федерации  – региона
- **РРР** – код района;
- **ГГГ** – код города;
- **ППП** код населенного пункта;
- **СССС** - код элемента планировочной структуры;
- **УУУУ** - код улицы;
- **ДДДД** (или **ЗЗЗЗ**).  ДДДД  тип и номер здания, сооружения, объекта незавершенного строительства в случае адресации домов. ЗЗЗЗ - номер земельного участка в случае адресации земельных участков;
- **ОООО** - тип и номер помещения в пределах здания, сооружения
Таким образом, каждому уровню классификации соответствует фасет кода.

**Пример 1**

Берем PLAINCODE (тот же CODE, но без двух последних цифр, обозначающих актуальность).

320180000580002 и разбиваем

REGIONCODE - 32 - Брянская область

AREACODE - 018 - Новозыбковский район

CITYCODE - 000 - нет

PLACECODE - 058 - с. Старый Вышков

STREETCODE - 0002 - ул. Ленина

**Пример 2**

320000040000081

REGIONCODE - 32 - Брянская область

AREACODE - 000 - района нет, город областного подчинения

CITYCODE - 004 - Новозыбков

PLACECODE - 000 - none

STREETCODE - 0081 - ул. Ленина

Послений параметр варьируется (STREETCODE / PLANCODE )

**Пример 3**

320050000010122


REGIONCODE - 32 - Брянская область

AREACODE - 005 - Дубровский район

CITYCODE - 000 - none

PLACECODE - 001 - рп Дубровка

PLANCODE - 0122 - гаражи по улице Ленина Блок 1

In [3]:
# структура датафреймов: data - полный датасет (только актуальные значения, LIVESTATUS - 1)
#                        work - рабочие сеты, дополненные и отфильтрованные 
#                        lenin - отфильтрованные по семантике

In [4]:
# читаем файл

dbf = DBF('C:/00_Projects/Lenin/FIAS/ADDROB50.DBF')
data = pd.DataFrame(dbf)

In [5]:
# фильтруем данные, оставляем только актуальные адреса LIVESTATUS = 1

data = data[data.LIVESTATUS == 1]

In [6]:
data.shape

(67567, 39)

In [7]:
# удаляем дубликаты (т.е. полностью идентичные записи)

data = data.drop_duplicates()

In [8]:
data.shape

(67567, 39)

In [9]:
# названия столбцов оригинального файла

all_columns = list(data)
print(all_columns)

['ACTSTATUS', 'AOGUID', 'AOID', 'AOLEVEL', 'AREACODE', 'AUTOCODE', 'CENTSTATUS', 'CITYCODE', 'CODE', 'CURRSTATUS', 'ENDDATE', 'FORMALNAME', 'IFNSFL', 'IFNSUL', 'NEXTID', 'OFFNAME', 'OKATO', 'OKTMO', 'OPERSTATUS', 'PARENTGUID', 'PLACECODE', 'PLAINCODE', 'POSTALCODE', 'PREVID', 'REGIONCODE', 'SHORTNAME', 'STARTDATE', 'STREETCODE', 'TERRIFNSFL', 'TERRIFNSUL', 'UPDATEDATE', 'CTARCODE', 'EXTRCODE', 'SEXTCODE', 'LIVESTATUS', 'NORMDOC', 'PLANCODE', 'CADNUM', 'DIVTYPE']


In [10]:
#  all_columns, сгруппированные по содержанию

#  Статус записи:
#  'ACTSTATUS', 'CURRSTATUS', 'LIVESTATUS', 'OPERSTATUS'

#  Дата записи:
#  'STARTDATE', 'ENDDATE', 'UPDATEDATE'

#  Название объекта:
#  'FORMALNAME', 'OFFNAME'

#  Тип объекта:
#  'SHORTNAME', 'AOLEVEL'

#  фасетный код

#  сам код: 'CODE', 'PLAINCODE'
#  'REGIONCODE', 'AREACODE', 'CITYCODE', 'PLACECODE', 'PLANCODE', 'STREETCODE',

#  Коды ОКАТО, ОКТМО, индексы, налоговая:
#  'OKATO', 'OKTMO',
#  'POSTALCODE',
#  'IFNSFL', 'IFNSUL', 'TERRIFNSFL', 'TERRIFNSUL',


#  'AOGUID', 'AOID', 'AUTOCODE', 'CENTSTATUS', 

#   'NEXTID', 'PARENTGUID',  

#  'PREVID',  

#  'CTARCODE', 'EXTRCODE', 'SEXTCODE', 'NORMDOC', 'CADNUM', 'DIVTYPE'

In [11]:
# отбираем необходимые столбцы для рабочего датафрейма

necessary_columns = ['AOLEVEL', 'FORMALNAME', 'OFFNAME', 'SHORTNAME',
                     'PLAINCODE', 'REGIONCODE', 'AREACODE',
                     'CITYCODE', 'PLACECODE', 'PLANCODE', 'STREETCODE',
                     'OKATO', 'OKTMO', 'POSTALCODE'
                    ]

In [12]:
# оставляем только нужные столбцы в новом фрейме work

work = data[necessary_columns]
work.columns = work.columns.str.lower()

In [13]:
# меняем типы столбцов для корректного слияния

work = work.astype({'aolevel': 'int32', 'regioncode': 'int32'})

In [14]:
work.isnull().sum()

aolevel       0
formalname    0
offname       0
shortname     0
plaincode     0
regioncode    0
areacode      0
citycode      0
placecode     0
plancode      0
streetcode    0
okato         0
oktmo         0
postalcode    0
dtype: int64

**Не верьте нулям!**

In [15]:
work.shape

(67567, 14)

In [16]:
# проверяем столбец на пустые значения

work[work.formalname == ''].shape

(0, 14)

In [17]:
# проверяем столбец на пустые значения

work[work.offname == ''].shape

(1392, 14)

Из-за глюка последних версия БД, в столбце OFFNAME много пропусков, в FORMALNAME - вроде все нормально. На всякий случай, сохраняем и off, и formal. Но основной - formalname.

**Удаляем лишнее для экономии ресурсов**

Оставляем только записи с уровнем (aolevel) - 1, 3, 4, 6, 7.

65, 90 -- в нашем случае только зашумляют картину.

In [18]:
work.shape

(67567, 14)

In [19]:
# оставляем только нужные записи

work = (work[
    (work.aolevel == 1) |
    (work.aolevel == 3) |
    (work.aolevel == 4) |
    (work.aolevel == 6) |
    (work.aolevel == 7) 
]
)

In [20]:
work.shape

(47410, 14)

In [21]:
work.dtypes

aolevel        int32
formalname    object
offname       object
shortname     object
plaincode     object
regioncode     int32
areacode      object
citycode      object
placecode     object
plancode      object
streetcode    object
okato         object
oktmo         object
postalcode    object
dtype: object

In [22]:
# смотрим уровни и что там внутри

level_df = work.aolevel.value_counts().rename_axis('level').reset_index(name='counts').sort_values('level')
level_df.head(10)

,level,counts
4,1,1
3,3,17
2,4,76
1,6,5994
0,7,41322


### 3 - уровень района ###

In [23]:
# выводим данные по каждому уровню, по ячейкам

(work[work.aolevel == 3]
 .shortname.value_counts()
 .rename_axis('element').reset_index(name='number')
 .sort_values('number', ascending=False)
)

,element,number
0,г,9
1,р-н,8


### 4 - уровень города ###

In [24]:
(work[work.aolevel == 4]
 .shortname.value_counts()
 .rename_axis('element').reset_index(name='number')
 .sort_values('number', ascending=False)
)

,element,number
0,г,65
1,с/п,4
2,г.,3
3,рп,3
4,пгт,1


### 6 - уровень поселка ###

In [25]:
(work[work.aolevel == 6]
 .shortname.value_counts()
 .rename_axis('element').reset_index(name='number')
 .sort_values('number', ascending=False)
)

,element,number
0,д,4747
1,п,541
2,с,534
3,рп,55
4,х,25
5,тер,15
6,мкр,15
7,кв-л,14
8,дп,13
9,снт,10


### 7 - уровень улицы ###

In [26]:
(work[work.aolevel == 7]
 .shortname.value_counts()
 .rename_axis('element').reset_index(name='number')
 .sort_values('number', ascending=False)
)

,element,number
0,ул,32413
1,пер,2589
2,проезд,2051
3,тер,826
4,туп,728
5,пр-д,560
6,гск,394
7,ш,346
8,км,248
9,кв-л,196


Сейчас в данных еще много лишнего, почистим позднее, после того, как сведем всё в один файл.

### Добавляем регион ###

In [27]:
# создаем датафрейм с названиями регионов и кодами ФИАС

reg_code = pd.read_csv('C:/00_Projects/Lenin/Datasets/03_Other/reg_code.csv')
reg_code.regioncode = reg_code.regioncode.astype('int32')

In [28]:
reg_code.dtypes

regioncode     int32
region        object
dtype: object

In [29]:
# добавляем регион

work = work.merge(reg_code, on='regioncode', how='left')

In [30]:
# проверяем

work.isnull().sum()

aolevel       0
formalname    0
offname       0
shortname     0
plaincode     0
regioncode    0
areacode      0
citycode      0
placecode     0
plancode      0
streetcode    0
okato         0
oktmo         0
postalcode    0
region        0
dtype: int64

In [31]:
work.head()

,aolevel,formalname,offname,shortname,plaincode,regioncode,areacode,citycode,placecode,plancode,streetcode,okato,oktmo,postalcode,region
0,4,Жуковский,Жуковский,г,50000005000,50,000,005,000,0000,0000,46425000000,46725000001,,Московская область
1,6,Одинцово,Одинцово,д,50000001012,50,000,001,012,0000,0000,46409000411,46709000511,,Московская область
2,4,Дубна,Дубна,г,50000003000,50,000,003,000,0000,0000,46418000000,46718000001,,Московская область
3,4,Котельники,Котельники,г,50000032000,50,000,032,000,0000,0000,46444000000,46739000,,Московская область
4,6,Мотякино,Мотякино,д,50000001211,50,000,001,211,0000,0000,46409000000,46709000001,142040,Московская область


### Добавляем муниципальный район ###

In [32]:
# отбираем районы и города и поселки регионального подчинения

mun_district = work[(
    (work.aolevel == 3)
 |
    (
        (work.aolevel == 4) &
        (work.areacode == '000' )) |
    (
        (work.aolevel == 6) &
        (work.areacode == '000') &
        (work.citycode == '000')
    )
)]

In [33]:
mun_district.shape

(71, 15)

In [34]:
# Функция для разметки данных
# Режет plaincode в соответствии с "муниципальным статусом" объекта

def cut_code(row):
    
    # муниципалитеты
    
    if row.aolevel == 3: 
        return row.plaincode[0:5]
    
    # городские округа (у г.о. areacode - 000)
    
    elif row.aolevel == 4:
        if row.areacode == '000':
            return row.plaincode[0:8]
        
        else:
            return row.plaincode[0:5]

    # поселки и улицы
    
    elif row.aolevel == 6 or row.aolevel == 7:
        
        if row.areacode != '000':
            return row.plaincode[0:5]
                
        # поселки которые имеют статус округов
        
        elif row.areacode == '000' and row.citycode == '000':
            return row.plaincode[0:11]            
        
        # все остальное (000 в areacode означает, что объект входит в состав г.о. или с.о.)
        # поэтому берем 8 цифр
        
        else:
            return row.plaincode[0:8]

In [35]:
# добавляем mun_code в датафрейм для слияния

mun_district['mun_code'] = mun_district.apply(cut_code, axis=1)

C:\Users\Boris\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [36]:
# оставляем нужные столбцы

mun_district = mun_district[['formalname', 'mun_code']]

In [37]:
# переименовываем

mun_district = mun_district.rename(columns={'formalname': 'mun_district'})

In [38]:
# добавляем mun_code в датафрейм

work['mun_code'] = work.apply(cut_code, axis=1)

In [39]:
# объединяем

work = work.merge(mun_district, on='mun_code', how='left')

In [40]:
work.shape

(47410, 17)

In [41]:
# проверка на NaN
# один NaN должен быть в mun_code (это регион)
# некоторое количество может быть в mun_district -- это следствия изменений и ошибок

work.isnull().sum()

aolevel          0
formalname       0
offname          0
shortname        0
plaincode        0
regioncode       0
areacode         0
citycode         0
placecode        0
plancode         0
streetcode       0
okato            0
oktmo            0
postalcode       0
region           0
mun_code         1
mun_district    18
dtype: int64

## Пропуски в mun_district ##

Часть ошибок можно исправить автоматом

Можно, но нужно. Функция слепа, слишком много ошибок. Лучше после заполнить пустые значения вручную, по кодам ОКТМО с https://rosstat.gov.ru/opendata/7708234640-oktmo

In [42]:
# заполняем пустые значения
work.fillna('null', inplace=True)

In [43]:
work[work.mun_district == 'null']

,aolevel,formalname,offname,shortname,plaincode,regioncode,areacode,citycode,placecode,plancode,streetcode,okato,oktmo,postalcode,region,mun_code,mun_district
21894,7,Полевой,Полевой,туп,500400000000057,50,040,000,000,0010,0057,46259000000,46659402,141100,Московская область,50040,null
23794,7,45-й,45-й,км,500000690000360,50,000,069,000,0656,0360,46455000000,46755000,,Московская область,50000069,null
29621,1,Московская,Московская область,обл,50000000000,50,000,000,000,0000,0000,46000000000,46000000,,Московская область,null,null
31342,7,Ягодная,Ягодная,ул,500500420840012,50,050,042,084,0004,0012,46487000456,46787000556,,Московская область,50050,null
31347,7,Фасадная,Фасадная,ул,500000452220002,50,050,045,222,0000,0002,46466000618,46766000718,143103,Московская область,50000,null
31368,7,Спасская,Спасская,ул,500500420840006,50,050,042,084,0004,0006,46487000456,46787000556,,Московская область,50050,null
31406,7,Истринская,Истринская,ул,500500420840007,50,050,042,084,0004,0007,46487000456,46787000556,,Московская область,50050,null
31415,7,Березовая,Березовая,ул,500500420840005,50,050,042,084,0004,0005,46487000456,46787000556,,Московская область,50050,null
31417,7,Изумрудный,Изумрудный,пр-кт,500500451590001,50,050,045,159,0002,0001,46466000453,46766000553,143121,Московская область,50050,null
31431,7,Артемьевская,Артемьевская,ул,500500420840010,50,050,042,084,0004,0010,46487000456,46787000556,,Московская область,50050,null


In [44]:
# примитивная функция для заполнения по ОКТМО (смотрит на другие строки с таким же ОКТМО)
# для городов регионов (Москвы, СПб, Севастополя) функция вместо м.о. добавляет населенные пункты (это неправильно, но пока так)

def add_mun_district(row):
    need_code = row.oktmo
    try:
        new_value = work[(
            (work.oktmo == need_code) &
            (work.mun_district != 'null')
        )].mun_district.to_list()[0]
        
    except:
        new_value = 'null'
        
    return new_value

In [45]:
# заводим временный датафрейм

temp_md = work[work.mun_district == 'null']

In [46]:
%%time

# добавляем столбец

temp_md['new_md'] = temp_md.apply(add_mun_district, axis=1)

Wall time: 337 ms


C:\Users\Boris\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [47]:
temp_md

,aolevel,formalname,offname,shortname,plaincode,regioncode,areacode,citycode,placecode,plancode,streetcode,okato,oktmo,postalcode,region,mun_code,mun_district,new_md
21894,7,Полевой,Полевой,туп,500400000000057,50,040,000,000,0010,0057,46259000000,46659402,141100,Московская область,50040,null,null
23794,7,45-й,45-й,км,500000690000360,50,000,069,000,0656,0360,46455000000,46755000,,Московская область,50000069,null,Одинцово
29621,1,Московская,Московская область,обл,50000000000,50,000,000,000,0000,0000,46000000000,46000000,,Московская область,null,null,Пущино
31342,7,Ягодная,Ягодная,ул,500500420840012,50,050,042,084,0004,0012,46487000456,46787000556,,Московская область,50050,null,Шаховская
31347,7,Фасадная,Фасадная,ул,500000452220002,50,050,045,222,0000,0002,46466000618,46766000718,143103,Московская область,50000,null,Руза
31368,7,Спасская,Спасская,ул,500500420840006,50,050,042,084,0004,0006,46487000456,46787000556,,Московская область,50050,null,Шаховская
31406,7,Истринская,Истринская,ул,500500420840007,50,050,042,084,0004,0007,46487000456,46787000556,,Московская область,50050,null,Шаховская
31415,7,Березовая,Березовая,ул,500500420840005,50,050,042,084,0004,0005,46487000456,46787000556,,Московская область,50050,null,Шаховская
31417,7,Изумрудный,Изумрудный,пр-кт,500500451590001,50,050,045,159,0002,0001,46466000453,46766000553,143121,Московская область,50050,null,Руза
31431,7,Артемьевская,Артемьевская,ул,500500420840010,50,050,042,084,0004,0010,46487000456,46787000556,,Московская область,50050,null,Шаховская


In [48]:
# оставляем нужные столбцы для слияния (на всякий случай 4)

temp_md = temp_md[['formalname', 'offname', 'plaincode', 'oktmo', 'new_md']]

In [49]:
# соединяем

work = work.merge(temp_md, on=['formalname', 'offname', 'plaincode', 'oktmo'], how='left')

In [50]:
work.head()

,aolevel,formalname,offname,shortname,plaincode,regioncode,areacode,citycode,placecode,plancode,streetcode,okato,oktmo,postalcode,region,mun_code,mun_district,new_md
0,4,Жуковский,Жуковский,г,50000005000,50,000,005,000,0000,0000,46425000000,46725000001,,Московская область,50000005,Жуковский,NaN
1,6,Одинцово,Одинцово,д,50000001012,50,000,001,012,0000,0000,46409000411,46709000511,,Московская область,50000001,Домодедово,NaN
2,4,Дубна,Дубна,г,50000003000,50,000,003,000,0000,0000,46418000000,46718000001,,Московская область,50000003,Дубна,NaN
3,4,Котельники,Котельники,г,50000032000,50,000,032,000,0000,0000,46444000000,46739000,,Московская область,50000032,Котельники,NaN
4,6,Мотякино,Мотякино,д,50000001211,50,000,001,211,0000,0000,46409000000,46709000001,142040,Московская область,50000001,Домодедово,NaN


In [51]:
# заполняем NaN

work.fillna('null', inplace=True)

In [52]:
# меняем null в mun_district 

work.loc[(work.mun_district == 'null'), 'mun_district'] = work.new_md

In [53]:
# проверяем

work[work.mun_district == 'null'].shape

(1, 18)

### Добавляем населенный пункт ###

In [54]:
# создаем датафрейм с городами и поселками

city = work[(
    (work.aolevel == 4) |
    (work.aolevel == 6)
)]

In [55]:
# оставляем нужные столбцы

city = city[['shortname', 'formalname', 'plaincode']]

In [56]:
# переименовываем

city = city.rename(columns={'shortname': 'city_type', 'formalname': 'city', 'plaincode': 'city_code'})

In [57]:
city

,city_type,city,city_code
0,г,Жуковский,50000005000
1,д,Одинцово,50000001012
2,г,Дубна,50000003000
3,г,Котельники,50000032000
4,д,Мотякино,50000001211
...,...,...,...
47377,д,Валуйки,50000066208
47387,д,Рузино,50000065138
47389,д,Колтышево,50000065052
47396,с,Спирово,50000066248


In [58]:
# проверяем

city.shape

(6070, 3)

In [59]:
# создаем столбец "код города" для слияния

work['city_code'] = work.plaincode.str[0:11]

In [60]:
%%time

# объединяем

work = work.merge(city, on='city_code', how='left')

Wall time: 237 ms


In [61]:
work.shape

(47410, 21)

In [62]:
# пустые значения -- улицы городов "прямого подчинения" (и ошибки с прошлого шага)

work.isnull().sum()

aolevel            0
formalname         0
offname            0
shortname          0
plaincode          0
regioncode         0
areacode           0
citycode           0
placecode          0
plancode           0
streetcode         0
okato              0
oktmo              0
postalcode         0
region             0
mun_code           0
mun_district       0
new_md             0
city_code          0
city_type       2621
city            2621
dtype: int64

In [63]:
# заполняем NaN

work.fillna('null', inplace=True)

In [64]:
# есть некоторое незначительное количество районов (это ошибки ФИАС)

work[work.city == 'null'].mun_district.value_counts()

Ногинск              646
Сергиев Посад        488
Наро-Фоминск         285
Воскресенск          254
Клин                 242
Орехово-Зуево        228
Дмитров              181
Одинцово             179
Истра                 94
Шаховская              9
Павлово-Посадский      3
Рузский                2
Пущино                 1
Ленинский              1
Красногорский          1
Руза                   1
Чеховский              1
Зарайский              1
Серпуховский           1
null                   1
Коломенский            1
Ступино                1
Name: mun_district, dtype: int64

In [65]:
work

,aolevel,formalname,offname,shortname,plaincode,regioncode,areacode,citycode,placecode,plancode,...,okato,oktmo,postalcode,region,mun_code,mun_district,new_md,city_code,city_type,city
0,4,Жуковский,Жуковский,г,50000005000,50,000,005,000,0000,...,46425000000,46725000001,,Московская область,50000005,Жуковский,null,50000005000,г,Жуковский
1,6,Одинцово,Одинцово,д,50000001012,50,000,001,012,0000,...,46409000411,46709000511,,Московская область,50000001,Домодедово,null,50000001012,д,Одинцово
2,4,Дубна,Дубна,г,50000003000,50,000,003,000,0000,...,46418000000,46718000001,,Московская область,50000003,Дубна,null,50000003000,г,Дубна
3,4,Котельники,Котельники,г,50000032000,50,000,032,000,0000,...,46444000000,46739000,,Московская область,50000032,Котельники,null,50000032000,г,Котельники
4,6,Мотякино,Мотякино,д,50000001211,50,000,001,211,0000,...,46409000000,46709000001,142040,Московская область,50000001,Домодедово,null,50000001211,д,Мотякино
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47405,7,Западная,Западная,ул,500000651910201,50,000,065,191,0000,...,46471000557,46771000657,141531,Московская область,50000065,Солнечногорск,null,50000065191,д,Редино
47406,7,Крайняя,Крайняя,ул,500000651910213,50,000,065,191,0000,...,46471000557,46771000657,141531,Московская область,50000065,Солнечногорск,null,50000065191,д,Редино
47407,7,Танковая,Танковая,ул,500000651910217,50,000,065,191,0000,...,46471000557,46771000657,141531,Московская область,50000065,Солнечногорск,null,50000065191,д,Редино
47408,7,Военная,Военная,ул,500000651910200,50,000,065,191,0000,...,46471000557,46771000657,141531,Московская область,50000065,Солнечногорск,null,50000065191,д,Редино


In [66]:
# берем city из mun_district

work.loc[(work.city == 'null'), 'city'] = work.mun_district

In [67]:
work[work.city == 'null'].shape

(1, 21)

In [68]:
work.shape

(47410, 21)

### Фильтруем данные ###

In [69]:
# переименовываем formalname в понятный street

work.rename(columns={'formalname': 'street'}, inplace=True)

In [70]:
# фильтруем по нужной семантике

lenin_work = work[(
    (work.street.str.contains('ленин', case=False)) |
    (work.street.str.contains('ульянов', case=False)) |
    (work.street.str.contains('ильич', case=False))
)]

### Муниципальные районы ###

In [72]:
# датафрейм муниципальных образований

work_districts = work[(
    (work.aolevel == 3) |
    (
        (work.aolevel == 4) &
        (work.areacode == '000')) |
    (
        (work.aolevel == 6) &
        (work.areacode == '000') &
        (work.citycode == '000'))
)]


# оставляем важные столбцы
work_districts = work_districts[['aolevel', 'shortname', 'mun_district', 'offname',
                                 'region',
                                 'okato', 'oktmo', 'postalcode']]

In [73]:
#  "ленинские" муниципальные районы и города регионального подчинения

lenin_districts = lenin_work[(
    (lenin_work.aolevel == 3) |
    (
        (lenin_work.aolevel == 4) &
        (lenin_work.areacode == '000')) |
    (
        (lenin_work.aolevel == 6) &
        (lenin_work.areacode == '000') &
        (lenin_work.citycode == '000'))
)]

# оставляем важные столбцы
lenin_districts = lenin_districts[['aolevel', 'shortname', 'mun_district', 'offname',
                                   'region',
                                   'okato', 'oktmo', 'postalcode']]

### Города и поселки ###

In [74]:
#  фильтруем города и поселки

work_cities = work[(
    (work.aolevel == 4) |
    (work.aolevel == 6) |
    (
        (work.aolevel == 3) &
        ((work.shortname == 'г') |
         (work.shortname == 'п'))
    )
)]


# оставляем важные столбцы
work_cities = work_cities[['aolevel', 'city_type', 'city', 'offname',
                           'mun_district', 'region',
                           'okato', 'oktmo', 'postalcode']]

In [75]:
#  фильтруем "ленинские" города и поселки

lenin_cities = lenin_work[(
    (lenin_work.aolevel == 4) |
    (lenin_work.aolevel == 6)
)]


# оставляем важные столбцы
lenin_cities = lenin_cities[['aolevel', 'city_type', 'city', 'offname',
                           'mun_district', 'region',
                           'okato', 'oktmo', 'postalcode']]

### Улицы ###

In [76]:
#  фильтруем улицы и территории

work_streets = work[(
    (work.aolevel == 7) |
    (work.aolevel == 65)
)]

# оставляем важные столбцы
work_streets = work_streets[['aolevel', 'shortname', 'street', 'offname',
                             'city_type', 'city', 'mun_district', 'region',
                             'okato', 'oktmo', 'postalcode']]

In [77]:
#  фильтруем "ленинские" улицы и территории

lenin_streets = lenin_work[(
    (lenin_work.aolevel == 7) |
    (lenin_work.aolevel == 65)
)]

# оставляем важные столбцы
lenin_streets = lenin_streets[['aolevel', 'shortname', 'street', 'offname',
                             'city_type', 'city', 'mun_district', 'region',
                             'okato', 'oktmo', 'postalcode']]

In [78]:
# записываем в файл (чтобы посмотреть, что вышло)

lenin_streets.to_excel('C:/Users/Boris/Desktop/lenin_streets.xlsx', encoding='utf-8', index=False)
work_streets.to_excel('C:/Users/Boris/Desktop/work_streets.xlsx', encoding='utf-8', index=False)   

In [79]:
work_streets.shape

(41322, 11)